In [3]:
!pip install -U \
  langchain \
  langchain-core \
  langchain-community \
  langchain-openai \
  faiss-cpu \
  openai \
  tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langgraph-prebuilt
    Found existing installation: langgraph-prebuilt 1.0.6
    Uninstalling langgraph-prebuilt-1.0.6:
      Successfully uninstalled langgraph-prebuilt-1.0.6
  Attempt

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="""
        Contextual compression is a technique used in retrieval-augmented
        generation systems. While many documents contain useful information,
        they often also include background explanations, historical notes,
        or marketing language. ContextualCompressionRetriever uses an LLM
        to extract only the parts of a document that are relevant to a user query.
        """,
        metadata={"source": "doc1"}
    ),
    Document(
        page_content="""
        FAISS is a popular vector database developed by Facebook AI Research.
        It supports efficient similarity search and clustering of dense vectors.
        In practical RAG pipelines, FAISS is often paired with retrievers.
        However, the retrieved documents may still contain unnecessary sections,
        which is where contextual compression becomes useful.
        """,
        metadata={"source": "doc2"}
    ),
    Document(
        page_content="""
        Traditional retrievers return full documents or chunks even if only
        a single sentence is relevant. Contextual compression solves this by
        selectively extracting relevant passages before sending them to the LLM.
        This helps reduce token usage and improve answer quality.
        """,
        metadata={"source": "doc3"}
    ),
    Document(
        page_content="""
        In many enterprise knowledge bases, documents are written by different
        teams and include inconsistent formatting, redundant explanations,
        and off-topic sections. ContextualCompressionRetriever helps clean
        these documents dynamically at query time.
        """,
        metadata={"source": "doc4"}
    ),
    Document(
        page_content="""
        Large language models can act as intelligent filters. When used as
        document compressors, they analyze retrieved text and keep only
        information that directly answers the user question.
        The rest of the content is discarded.
        """,
        metadata={"source": "doc5"}
    ),
    Document(
        page_content="""
        Compression retrievers are especially helpful when documents are long
        or when chunking alone is not sufficient. Instead of relying only on
        embedding similarity, an LLM evaluates semantic relevance in context.
        """,
        metadata={"source": "doc6"}
    ),
    Document(
        page_content="""
        Contextual compression introduces additional LLM calls, which can
        increase latency and cost. For this reason, it is typically used
        after an initial retrieval step rather than as a standalone retriever.
        """,
        metadata={"source": "doc7"}
    ),
    Document(
        page_content="""
        When users ask focused questions, such as asking for a definition
        or a specific mechanism, contextual compression can remove
        introductory sections and examples that are not directly relevant.
        """,
        metadata={"source": "doc8"}
    ),
    Document(
        page_content="""
        In contrast to reranking, which orders documents by relevance,
        contextual compression modifies document content itself by
        extracting only the most useful spans of text.
        """,
        metadata={"source": "doc9"}
    ),
    Document(
        page_content="""
        A common pattern in RAG systems is: retrieve broadly, then compress.
        This allows the system to maintain high recall while still providing
        concise, high-signal context to the language model.
        """,
        metadata={"source": "doc10"}
    ),
]


In [ ]:
embedding_model=OpenAIEmbeddings()
vectorstore=FAISS.from_documents(documents,embedding_model)


In [ ]:
base_retriever=vectorstore.as_retriever(search_kwargs={"k":5})


In [ ]:
#Set up the compressor using an LLM
llm=ChatOpenAI(temperature=0)
compressor=LLMChainExtractor.from_llm(llm)

In [ ]:
compression_retriever=ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

In [ ]:
#Query the retriever
query="What is photosynthesis?"
compressed_docs=compression_retriever.invoke(query)